In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import yaml
import logging
from ionbeam.sources.cima import CIMA_API

with open("/Users/math/git/IonBeam_bundle/IonBeam/config/secrets.yaml") as f:
    secrets = yaml.safe_load(f)

api = CIMA_API(
    credentials = secrets["ACRONET"],
    cache_file = "./acronet_cache.pickle",
    logLevel = logging.DEBUG,
)



In [3]:
next(s for s in api.stations.values())

Station(name='Monte Santa Croce', id='monte_santa_croce', lat=44.148647, lon=9.679447, sensors={'PLUVIOMETRO': UniqueSensor(unit='mm', id='-1937152391_2'), 'TERMOMETRO': UniqueSensor(unit='°C', id='210331261_2'), 'IGROMETRO': UniqueSensor(unit='%', id='210331262_2'), 'DIREZIONEVENTO': UniqueSensor(unit='Degrees', id='210331263_2'), 'ANEMOMETRO': UniqueSensor(unit='m/s', id='210331264_2'), 'BAROMETRO': UniqueSensor(unit='hPa', id='210331268_2'), 'BATTERIA': UniqueSensor(unit='V', id='210331258_2'), 'TERMOMETRO_INTERNA': UniqueSensor(unit='°C', id='210331267_2'), 'DIREZIONEVENTO_RAFFICA': UniqueSensor(unit='Degrees', id='210331265_2'), 'ANEMOMETRO_RAFFICA': UniqueSensor(unit='m/s', id='210331266_2'), 'SIGNAL_STRENGTH': UniqueSensor(unit='CSQ', id='210331260_2')})

In [4]:
next(iter(api.sensors.values()))

GenericSensor(unit='mm', name='PLUVIOMETRO', translation='RAIN_GAUGE')

In [5]:
# construct sensor_id to station, sensor mapping
sensor_id_to_station = {
    sensor.id : (station.name, sensor_name)
    for station in api.stations.values()
    for sensor_name, sensor in station.sensors.items()
}
sensor_id_to_station

{'-1937152391_2': ('Monte Santa Croce', 'PLUVIOMETRO'),
 '210331261_2': ('Monte Santa Croce', 'TERMOMETRO'),
 '210331262_2': ('Monte Santa Croce', 'IGROMETRO'),
 '210331263_2': ('Monte Santa Croce', 'DIREZIONEVENTO'),
 '210331264_2': ('Monte Santa Croce', 'ANEMOMETRO'),
 '210331268_2': ('Monte Santa Croce', 'BAROMETRO'),
 '210331258_2': ('Monte Santa Croce', 'BATTERIA'),
 '210331267_2': ('Monte Santa Croce', 'TERMOMETRO_INTERNA'),
 '210331265_2': ('Monte Santa Croce', 'DIREZIONEVENTO_RAFFICA'),
 '210331266_2': ('Monte Santa Croce', 'ANEMOMETRO_RAFFICA'),
 '210331260_2': ('Monte Santa Croce', 'SIGNAL_STRENGTH'),
 '-1937156895_2': ('Campo Sportivo Bajardo', 'PLUVIOMETRO'),
 '210330148_2': ('Campo Sportivo Bajardo', 'TERMOMETRO'),
 '210330149_2': ('Campo Sportivo Bajardo', 'IGROMETRO'),
 '210330150_2': ('Campo Sportivo Bajardo', 'DIREZIONEVENTO'),
 '210330151_2': ('Campo Sportivo Bajardo', 'ANEMOMETRO'),
 '210330155_2': ('Campo Sportivo Bajardo', 'BAROMETRO'),
 '210326754_2': ('Campo Spor

In [6]:
from functools import cached_property, reduce
import pandas as pd
from collections import defaultdict
import numpy as np
from datetime import datetime, timedelta

start_date = datetime.now()- timedelta(minutes = 30)
end_date = datetime.now() - timedelta(minutes = 25)

data_by_station = defaultdict(list)

for sensor_class in api.sensors.keys():
    print(f"getting {sensor_class}")
    r = api.get(
            api.api_url + f"sensors/data/{sensor_class}/all",
            params={
            "from": start_date.strftime("%Y%m%d%H%M"),
            "to": end_date.strftime("%Y%m%d%H%M"),
            "aggr": 60,
            "date_as_string": True,
    })
    
    # Go through all the returned data and group it by station
    for data in r.json():
        print(data)
        break
        if data["sensorId"] not in sensor_id_to_station:
            # print(data["sensorId"])
            continue
        
        station_name, sensor_class = sensor_id_to_station[data["sensorId"]]
        station = api.stations[station_name]
        sensor = api.sensors[sensor_class]
        data["sensor_class"] = sensor_class
        data["sensor_unit"] = sensor.unit
        data_by_station[station_name].append(data)

getting PLUVIOMETRO
{'sensorId': '-1937152652_2', 'timeline': ['202412191219', '202412191220', '202412191221', '202412191222', '202412191223', '202412191224'], 'values': [0.0, 0.0, 0.0, 0.1, 0.0, 0.0]}
getting TERMOMETRO
{'sensorId': '210331008_2', 'timeline': ['202412191219', '202412191220', '202412191221', '202412191222', '202412191223', '202412191224'], 'values': [11.5, 11.5, 11.5, 11.400001, 11.400001, 11.400001]}
getting IGROMETRO
{'sensorId': '210330236_2', 'timeline': ['202412191219', '202412191220', '202412191221', '202412191222', '202412191223', '202412191224'], 'values': [66.200005, 66.700005, 67.0, 67.1, 67.0, 66.6]}
getting DIREZIONEVENTO
{'sensorId': '210331018_2', 'timeline': ['202412191219', '202412191220', '202412191221', '202412191222', '202412191223', '202412191224'], 'values': [178.6, 151.6, 164.1, 275.30002, 179.40001, 172.7]}
getting ANEMOMETRO
{'sensorId': '210796574_2', 'timeline': ['202412191219', '202412191220', '202412191221', '202412191222', '202412191223', '

In [7]:
# Go through each station group and convert it to a single dataframe for that station
station_dfs = []
for station_name, data in data_by_station.items():
    print(station_name)
    sensor_dfs = []
    for d in data:
        # print(d)
        col_name = f"{d['sensor_class']} [{d['sensor_unit']}]"
        single_df = pd.DataFrame({
            col_name : d["values"],
            "time": pd.to_datetime(d["timeline"], format="%Y%m%d%H%M", utc = True),
        })

        # Values below -9000 (specifically -9998.0 but somehow sometimes others?)
        # are used as a NaN sentinel value by Acronet
        # Filter these rows out
        single_df = single_df[single_df[col_name] != -9998.0]
        single_df = single_df[~single_df[col_name].isnull()]
        
        # If this sensor has any data
        if not single_df[col_name].isnull().all():
            sensor_dfs.append(single_df)

    # If this station has at least one sensor with data
    if sensor_dfs:
        df = reduce(
            lambda left, right: pd.merge(left, right, on=["time"], how="outer"),
            sensor_dfs
        )
        df["station_id"] =  api.stations[station_name].id
        df["station_name"] = station_name
        
        station_dfs.append(df)
        
station_dfs[0]

IndexError: list index out of range

In [ ]:
r.json()

In [ ]:
from datetime import datetime, timedelta, UTC

def get_all_data_for_sensor_class(api, sensor_class, start_date, end_date, aggregation_time_seconds: int = 60,):
    r = api.get(
        api.api_url + f"sensors/data/{sensor_class}/all",
        params={
        "from": start_date.strftime("%Y%m%d%H%M"),
        "to": end_date.strftime("%Y%m%d%H%M"),
        "aggr": aggregation_time_seconds,
        "date_as_string": True,
    },
    )
    sensor_data = r.json()
    sensor_data

def get_all_data(api, start_date: datetime,
        end_date: datetime,
        aggregation_time_seconds: int = 60):

    for sensor in api.sensors.values():
        print(f"Getting {sensor.translation}")
        data = get_all_data_for_sensor_class(api, sensor.name, start_date, end_date, aggregation_time_seconds)
        print(data[0])
        break

get_all_data(api, 
        start_date = datetime.now(),
        end_date = datetime.now() - timedelta(minutes = 5)
            )

In [ ]:
stations = defaultdict(list)

for sensor_class in sensor_classes:
    print(f"getting {sensor_class}")
    r = api.get(
        api.api_url + f"sensors/list/{sensor_class}/",
        params = dict(
            stationgroup="ComuneLive%IChange"
        )
    )
    for sensor in r.json():
        stations[sensor["name"]].append({
            'id': sensor["id"]
            'lat': 44.148647,
            'lng': 9.679447,
  'mu': 'mm'},
        })

In [ ]:
from matplotlib import pyplot as plt
from ionbeam.core.time import (
    TimeSpan,
    collapse_time_spans_upto_max,
    split_df_into_time_chunks,
    split_time_interval_into_chunks,
)

def plot_spans(spans, ax = None):
    if ax is None: f, ax = plt.subplots(1, 1, figsize = (10, 2))
    
    for ts in spans:
            ax.axvline(ts.start, linestyle = "dotted", color = "green")
            ax.axvline(ts.end, linestyle = "solid", color = "orange")
            ax.axvspan(ts.start, ts.end, alpha = 0.1, color = "green")

now = datetime.now()
start  = now - timedelta(minutes = 60)
end = now
granularity = timedelta(minutes = 5)

time_span = TimeSpan(start, end)
ingested = [
    TimeSpan(now - timedelta(minutes = 55), now - timedelta(minutes = 50)),
    TimeSpan(now - timedelta(minutes = 35), now - timedelta(minutes = 30)),
]
plot_spans(ingested)

In [ ]:
to_ingest = time_span.remove(ingested)


In [ ]:
maximum_request_size = timedelta(minutes = 15)
collapsed_time_spans = collapse_time_spans_upto_max(time_spans, maximum_request_size)

f, axes = plt.subplots(2, 1, figsize = (10, 2), sharex = True)
for full_time_span, time_span_chunks in collapsed_time_spans:
    ax = axes[0]
    for ts in time_span_chunks:
        ax.axvline(ts.start, linestyle = "dotted", color = "green")
        ax.axvline(ts.end, linestyle = "solid", color = "orange")
        ax.axvspan(ts.start, ts.end, alpha = 0.1, color = "green")

    ax = axes[1]
    ts = full_time_span
    ax.axvline(ts.start, linestyle = "dotted", color = "green")
    ax.axvline(ts.end, linestyle = "solid", color = "orange")
    ax.axvspan(ts.start, ts.end, alpha = 0.1, color = "green")

In [ ]:
from ionbeam.sources import AcronetSource
from ionbeam.core import parse_single_action, parse_config
from pathlib import Path

config_file = Path("~/git/IonBeam_bundle/IonBeam/config").expanduser()
action_yaml = """
class: AcronetSource
"""

config, source = parse_single_action(config_file, action_yaml, 
                        offline = False,
                        environment  = "local")

source